This Jupyter file can be run in Google colab environment. The cells includes are sorted so just run one follows by the one above. The code requires files: "test.csv.zip", "train.csv.zip" and "suffled_images.csv", the three files should be put into your working directory. The models I used are CNN and LSTM, I uploaded two saved models in Learn as well. For RNN the model is named "char_rnn_lstm_classification_model", for CNN it is named "best_model". And I use ensembled learning to concatenate their output probability using simple neural network, this model is also saved as "ensembled_model.pt".

In [162]:
import tensorflow as tf                                          # 1)
import numpy as np                                               # 2)
import keras                                                     # 3)
import pandas as pd                                              # 4)
from keras.models import Sequential                              # 5)
from keras.layers import Dense, Dropout, Activation, Flatten     # 6)
from keras.datasets import cifar10                               # 7)
from keras.layers import Conv2D, MaxPooling2D                    # 8)
from keras.preprocessing.image import ImageDataGenerator         # 9)
import matplotlib.pyplot as plt                                  # 10)
from keras.utils import np_utils                                 # 11)
from keras.layers.normalization import BatchNormalization        # 12)

In [2]:
import zipfile
with zipfile.ZipFile("train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [3]:
import zipfile
with zipfile.ZipFile("test.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [4]:
import zipfile
with zipfile.ZipFile("suffled-images.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

**1)use CNN to classiy images**

In [337]:
import os
dir=os.listdir('shuffled-images/')
from tkinter import Tcl
dir_sort=Tcl().call('lsort', '-dict', dir)
print(dir_sort[:10])
len(dir_sort)


('1163.jpg', '1164.jpg', '1165.jpg', '1525.jpg', '1526.jpg', '1528.jpg', '1529.jpg', '1530.jpg', '1531.jpg', '1532.jpg')


43255

In [148]:
train=pd.read_csv('train.csv',header=0)
print(train.columns)
train.head#21627

Index(['id', 'category', 'gender', 'baseColour', 'season', 'usage',
       'noisyTextDescription'],
      dtype='object')


<bound method NDFrame.head of           id    category  gender baseColour  season   usage  \
0      36274     Scarves   Women       Grey  Summer  Casual   
1      15129  Flip Flops  Unisex      Green  Summer  Casual   
2      58976     Topwear   Women        Red  Summer  Ethnic   
3      32922      Sandal     Men      Brown  Summer  Casual   
4      29561     Topwear   Women       Pink    Fall  Ethnic   
...      ...         ...     ...        ...     ...     ...   
21622   5435   Innerwear     Men      Black  Summer  Casual   
21623  27880       Belts     Men      Brown  Summer  Casual   
21624  38385       Shoes   Women      Beige  Winter  Casual   
21625  34853     Watches     Men      Steel  Winter  Casual   
21626  41407     Topwear     Men      White  Summer  Casual   

                                    noisyTextDescription  
0      Femella Women Ankle-Length Grey AQ-S800WD-1EVD...  
1                  Converse Unisex Casual Skirts Slipper  
2                              Velia

In [149]:
test=pd.read_csv('test.csv',header=0)
print(test.columns)
test.head#21628

Index(['id', 'gender', 'baseColour', 'season', 'usage',
       'noisyTextDescription'],
      dtype='object')


<bound method NDFrame.head of           id  gender baseColour  season   usage  \
0      26266     Men      Black  Summer  Casual   
1      22134   Women      Green  Summer  Casual   
2      28358   Women      Black  Winter  Casual   
3      15554     Men      Black    Fall  Casual   
4      53408   Women     Purple  Summer  Casual   
...      ...     ...        ...     ...     ...   
21623  39737     Men      Brown  Winter  Casual   
21624  57477   Women      Black  Summer  Casual   
21625  22312     Men      White    Fall  Sports   
21626  54105   Women        Red  Spring  Casual   
21627  14080  Unisex        Red  Summer  Casual   

                                    noisyTextDescription  
0                      Chromozome Men Black Fashion Vest  
1                       Elle Women Green Color Clash Top  
2                   Baggit Women Chotu Mayur Black Palms  
3      Greensboro Colors Of Ap Men Latnam Black Casua...  
4                        Fish White Smooth daddy (SH100)  
...

In [150]:
from os import listdir
from matplotlib import image
# load all images in a directory
loaded_images_train = list()
for filename in train['id']:
    #print(filename)
  # load image
    img_data = image.imread('shuffled-images/' + str(filename) + ".jpg")
    # store loaded image
    loaded_images_train.append(img_data)
   # print('> loaded %s %s' % (filename, img_data.shape))

In [151]:
from os import listdir
from matplotlib import image
# load all images in a directory
loaded_images_test = list()
for filename in test['id']:
    #print(filename)
  # load image
    img_data = image.imread('shuffled-images/' + str(filename) + ".jpg")
    # store loaded image
    loaded_images_test.append(img_data)
   # print('> loaded %s %s' % (filename, img_data.shape))

In [152]:
batch_size = 256      #batch size
num_classes = 27      #number of classes
epochs = 100          #epoch size

In [153]:
label=train['category'].tolist()

In [154]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label_list=train['category'].tolist()
le.fit(label_list)
transfomed_label =le.transform(label_list)
print(transfomed_label)
print(transfomed_label.shape)

[20  8 24 ... 21 26 24]
(21627,)


all categories are sorted alphabetically:

In [155]:
all_categories=le.inverse_transform(range(27)).tolist()

In [156]:
all_categories

['Accessories',
 'Apparel Set',
 'Bags',
 'Belts',
 'Bottomwear',
 'Cufflinks',
 'Dress',
 'Eyewear',
 'Flip Flops',
 'Fragrance',
 'Free Gifts',
 'Headwear',
 'Innerwear',
 'Jewellery',
 'Lips',
 'Loungewear and Nightwear',
 'Makeup',
 'Nails',
 'Sandal',
 'Saree',
 'Scarves',
 'Shoes',
 'Socks',
 'Ties',
 'Topwear',
 'Wallets',
 'Watches']

In [157]:
list(le.inverse_transform(transfomed_label))[:10]

['Scarves',
 'Flip Flops',
 'Topwear',
 'Sandal',
 'Topwear',
 'Bags',
 'Socks',
 'Topwear',
 'Topwear',
 'Shoes']

Split last 4325 obs into validation set, remaining set as training set:

In [158]:
x_train=np.array(loaded_images_train)
y_train=np.array(transfomed_label)
x_test=np.array(loaded_images_test)
x_val=x_train[-4325:]
y_val=y_train[-4325:]
x_train=x_train[:-4325]
y_train=y_train[:-4325]
print("x_train is ",x_train.shape[0])
print("x_val is ",x_val.shape[0])
print("x_test is ",x_test.shape[0])


x_train is  17302
x_val is  4325
x_test is  21628


In [159]:
y_train = np_utils.to_categorical(y_train, num_classes)#convert train labels to one hot encoding
y_val = np_utils.to_categorical(y_val, num_classes)  #convert test labels to one hot encoding

In [160]:
#Typecast data samples to flot32. It is usefull when using GPU.
x_train = x_train.astype('float32')   
x_test = x_test.astype('float32')
x_val=x_val.astype('float32')
# Scale pixles of data samples between 0 and 1.
x_train /= 255
x_test /= 255
x_val /= 255

In [365]:
# define a function to train with or without data augmentation:
def data_aug (save_best_model):
  if not data_augmentation:
      print('Not using data augmentation.')
      history = model.fit(x_train, y_train,verbose=1,
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(x_val,y_val),
                          shuffle=True,
                          callbacks=[save_best_model])

  # train with data augmentation
  else:
      print('Using real-time data augmentation.')
      # This will do preprocessing and realtime data augmentation:
      datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          zca_epsilon=1e-06,  # epsilon for ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          # randomly shift images horizontally (fraction of total width)
          width_shift_range=0.1,
          # randomly shift images vertically (fraction of total height)
          height_shift_range=0.1,
          shear_range=0.,  # set range for random shear
          zoom_range=0.,  # set range for random zoom
          channel_shift_range=0.,  # set range for random channel shifts
          # set mode for filling points outside the input boundaries
          fill_mode='nearest',
          cval=0.,  # value used for fill_mode = "constant"
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False,  # randomly flip images
          # set rescaling factor (applied before any other transformation)
          rescale=None,
          # set function that will be applied on each input
          preprocessing_function=None,
          # image data format, either "channels_first" or "channels_last"
          data_format=None,
          # fraction of images reserved for validation (strictly between 0 and 1)
          validation_split=0.0)

      # Compute quantities required for feature-wise normalization
      # (std, mean, and principal components if ZCA whitening is applied).
      datagen.fit(x_train)

      # Fit the model on the batches generated by datagen.flow().
      history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),verbose=1,
                          steps_per_epoch=math.ceil(x_train.shape[0]/batch_size),
                          epochs=epochs,
                          validation_data=(x_val,y_val),
                          callbacks=[save_best_model])
  return(history)   

In [164]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import math
import matplotlib.pyplot as plt

In [366]:
# libraries (do not import additional libraries)


# parameters for this script
batch_size = 32
num_classes = 27
epochs = 20
data_augmentation = True


# Define a convolutional neural network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Compile the model before using it
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 80, 60, 32)        896       
_________________________________________________________________
activation_15 (Activation)   (None, 80, 60, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 80, 60, 32)        128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 80, 60, 32)        9248      
_________________________________________________________________
activation_16 (Activation)   (None, 80, 60, 32)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 80, 60, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 40, 30, 32)       

In [21]:
# create a callback that will save the best model while training
save_best_model = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# call function "data_aug" to train without data augmentation:
history=data_aug(save_best_model)


# Evaluate the best model saved (i.e., model with best validation accuracy) on the test set
saved_model = load_model('best_model.h5')

Using real-time data augmentation.
Epoch 1/20
541/541 [==============================] - 401s 741ms/step - loss: 0.4590 - accuracy: 0.8715 - val_loss: 0.4700 - val_accuracy: 0.8719

Epoch 00001: val_accuracy improved from -inf to 0.87191, saving model to best_model.h5
Epoch 2/20
541/541 [==============================] - 401s 742ms/step - loss: 0.4548 - accuracy: 0.8725 - val_loss: 0.4074 - val_accuracy: 0.8990

Epoch 00002: val_accuracy improved from 0.87191 to 0.89896, saving model to best_model.h5
Epoch 3/20
541/541 [==============================] - 400s 740ms/step - loss: 0.4381 - accuracy: 0.8757 - val_loss: 0.4383 - val_accuracy: 0.8946

Epoch 00003: val_accuracy did not improve from 0.89896
Epoch 4/20
541/541 [==============================] - 400s 739ms/step - loss: 0.4380 - accuracy: 0.8764 - val_loss: 0.3654 - val_accuracy: 0.9089

Epoch 00004: val_accuracy improved from 0.89896 to 0.90890, saving model to best_model.h5
Epoch 5/20
541/541 [==============================] - 4

In [43]:
epochs=5
# create a callback that will save the best model while training
save_best_model = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# call function "data_aug" to train without data augmentation:
history=data_aug(save_best_model)


# Evaluate the best model saved (i.e., model with best validation accuracy) on the test set
saved_model = load_model('best_model.h5')

Using real-time data augmentation.
Epoch 1/5
541/541 [==============================] - 394s 727ms/step - loss: 0.3536 - accuracy: 0.9026 - val_loss: 0.3455 - val_accuracy: 0.9131

Epoch 00001: val_accuracy improved from -inf to 0.91306, saving model to best_model.h5
Epoch 2/5
541/541 [==============================] - 396s 732ms/step - loss: 0.3467 - accuracy: 0.9002 - val_loss: 0.3732 - val_accuracy: 0.9077

Epoch 00002: val_accuracy did not improve from 0.91306
Epoch 3/5
541/541 [==============================] - 394s 729ms/step - loss: 0.3403 - accuracy: 0.9041 - val_loss: 0.3671 - val_accuracy: 0.9059

Epoch 00003: val_accuracy did not improve from 0.91306
Epoch 4/5
541/541 [==============================] - 395s 730ms/step - loss: 0.3377 - accuracy: 0.9042 - val_loss: 0.3364 - val_accuracy: 0.9156

Epoch 00004: val_accuracy improved from 0.91306 to 0.91561, saving model to best_model.h5
Epoch 5/5
541/541 [==============================] - 395s 729ms/step - loss: 0.3375 - accuracy

In [44]:
epochs=5
# create a callback that will save the best model while training
save_best_model = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# call function "data_aug" to train without data augmentation:
history=data_aug(save_best_model)


# Evaluate the best model saved (i.e., model with best validation accuracy) on the test set
saved_model = load_model('best_model.h5')

Using real-time data augmentation.
Epoch 1/5
541/541 [==============================] - 394s 728ms/step - loss: 0.3364 - accuracy: 0.9064 - val_loss: 0.3356 - val_accuracy: 0.9119

Epoch 00001: val_accuracy improved from -inf to 0.91191, saving model to best_model.h5
Epoch 2/5
541/541 [==============================] - 392s 725ms/step - loss: 0.3349 - accuracy: 0.9058 - val_loss: 0.3558 - val_accuracy: 0.9140

Epoch 00002: val_accuracy improved from 0.91191 to 0.91399, saving model to best_model.h5
Epoch 3/5
541/541 [==============================] - 395s 729ms/step - loss: 0.3254 - accuracy: 0.9058 - val_loss: 0.3868 - val_accuracy: 0.9073

Epoch 00003: val_accuracy did not improve from 0.91399
Epoch 4/5
541/541 [==============================] - 394s 728ms/step - loss: 0.3202 - accuracy: 0.9072 - val_loss: 0.3141 - val_accuracy: 0.9214

Epoch 00004: val_accuracy improved from 0.91399 to 0.92139, saving model to best_model.h5
Epoch 5/5
541/541 [==============================] - 395s 7

In [45]:
epochs=5
# create a callback that will save the best model while training
save_best_model = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# call function "data_aug" to train without data augmentation:
history=data_aug(save_best_model)


# Evaluate the best model saved (i.e., model with best validation accuracy) on the test set
saved_model = load_model('best_model.h5')

Using real-time data augmentation.
Epoch 1/5
541/541 [==============================] - 396s 732ms/step - loss: 0.3185 - accuracy: 0.9075 - val_loss: 0.3290 - val_accuracy: 0.9177

Epoch 00001: val_accuracy improved from -inf to 0.91769, saving model to best_model.h5
Epoch 2/5
541/541 [==============================] - 397s 734ms/step - loss: 0.3193 - accuracy: 0.9102 - val_loss: 0.3706 - val_accuracy: 0.9064

Epoch 00002: val_accuracy did not improve from 0.91769
Epoch 3/5
541/541 [==============================] - 395s 730ms/step - loss: 0.3139 - accuracy: 0.9108 - val_loss: 0.3660 - val_accuracy: 0.9128

Epoch 00003: val_accuracy did not improve from 0.91769
Epoch 4/5
541/541 [==============================] - 393s 727ms/step - loss: 0.3083 - accuracy: 0.9129 - val_loss: 0.3477 - val_accuracy: 0.9145

Epoch 00004: val_accuracy did not improve from 0.91769
Epoch 5/5
541/541 [==============================] - 395s 730ms/step - loss: 0.3150 - accuracy: 0.9114 - val_loss: 0.3159 - val_a

In [165]:
model_CNN = load_model('best_model.h5')

Predict on training set:

In [166]:
import pandas as pd
prd_train = model_CNN.predict(x_train) 
prd_y_train = np.argmax(prd_train, axis=1)
label_train = list(le.inverse_transform(prd_y_train))

prd_val = model_CNN.predict(x_val) 
prd_y_val = np.argmax(prd_val, axis=1)
label_val= list(le.inverse_transform(prd_y_val))

In [167]:
# These lines are used for converting one hot coding back to the original label form.
# This function of trained model predicts the label of its input data
prd_test = model_CNN.predict(x_test) 
prd_y_test = np.argmax(prd_test, axis=1)
label_test = list(le.inverse_transform(prd_y_test))

**2)Use LSTM to classify noist text based on sentence to tensor  function (break down into letters, digits and punctuation):**

In [178]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [179]:
#train['combine_text']=train["noisyTextDescription"].astype(str)+' '+train["gender"].astype(str)+' '+train['baseColour'].astype(str)+' '+train['season'].astype(str)+' '+train['usage'].astype(str)
train['combine_text']=train["noisyTextDescription"].astype(str)
train['combine_text'][:10]
validate_data=np.array(train['combine_text'][-4325:]).tolist()
validate_y=np.array(train['category'][-4325:]).tolist()
training_data=np.array(train['combine_text'][:-4325]).tolist()
training_y=np.array(train['category'][:-4325]).tolist()
print(len(validate_data))
len(training_data)

4325


17302

In [180]:
import numpy as np
train_data_dict={}
for i in range(len(training_data)):
    if training_y[i] not in train_data_dict:
        train_data_dict[training_y[i]]=[training_data[i]]
    else:
        train_data_dict[training_y[i]].append(training_data[i])
print(len(train_data_dict))
train_data=train_data_dict

27


In [181]:
val_data_dict={}
for i in range(len(validate_data)):
    if validate_y[i] not in val_data_dict:
        val_data_dict[validate_y[i]]=[validate_data[i]]
    else:
        val_data_dict[validate_y[i]].append(validate_data[i])
print(len(val_data_dict))
val_data=val_data_dict

27


In [185]:
import unicodedata
import string
import math

all_letters = string.ascii_letters + string.digits + string.punctuation
#all_letters = string.ascii_letters + string.digits
n_letters = len(all_letters) 
n_categories=len(all_categories)
n_letters

94

In [186]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

def sentenceToTensor(sentence):
    tensor = torch.zeros(len(sentence), 1, n_letters)
    for li, letter in enumerate(sentence):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor
         

#print(letterToTensor('J')[0])

#print(lineToTensor('Jones 12&'))

#print(sentenceToTensor('Joes Harr'))

In [187]:
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_LSTM, self).__init__()

        self.hidden_size = hidden_size

        self.LSTM=nn.LSTMCell(input_size,hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        hidden, cell = self.LSTM(input, (hidden,cell))
        combined = torch.cat((input, hidden), 1)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output,hidden,cell

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn_lstm = RNN_LSTM(n_letters, n_hidden, n_categories)  

In [188]:
input = sentenceToTensor('Annetta Harry')
hidden = torch.zeros(1, n_hidden)
cell=torch.zeros(1, n_hidden)
output,hidden,cell= rnn_lstm(input[0], hidden,cell)
print(output)

tensor([[-3.2939, -3.2215, -3.2752, -3.3243, -3.3977, -3.2587, -3.2623, -3.3840,
         -3.3781, -3.2166, -3.3842, -3.2898, -3.2512, -3.3715, -3.3812, -3.3230,
         -3.2490, -3.2936, -3.3170, -3.2395, -3.2764, -3.2814, -3.2662, -3.2901,
         -3.2734, -3.2821, -3.2428]], grad_fn=<LogSoftmaxBackward>)


In [189]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Fragrance', 9)


In [190]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(train_data[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = sentenceToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Bags / line = Ivory Roshe Cheeky G260 Furry Foldover Sling Bag
category = Makeup / line = Streetwear PowerCat Divine Whitening INSP Compact
category = Topwear / line = Puma Men Graphic White Tshirts
category = Saree / line = Golden Green & Purple Wedding Collection Sari
category = Watches / line = Q&Q Men Black Dial Watch
category = Lips / line = Lotus Herbals Moistpetals Cocoa SGW-300H-1AVDR Matka 163
category = Wallets / line = Squid Women Black Figio
category = Nails / line = Piper NE3039SM01 Fast Metal-Eyes Aston A633 Crush Magenta Nail Polish 25
category = Innerwear / line = N.Masti Women Peach-Coloured Briefs PCLR02
category = Wallets / line = United Colors of Benetton Women Basice Wallet


In [191]:
criterion = nn.NLLLoss()

In [192]:
def train_LSTM(category_tensor, line_tensor):
    hidden = rnn_lstm.initHidden()
    cell= rnn_lstm.initHidden()
    # reset gradient
    rnn_lstm.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output,hidden,cell = rnn_lstm(line_tensor[i], hidden, cell)
    loss = criterion_lstm(output, category_tensor)

    # compute gradient by backpropagation
    loss.backward()

    # update parameters
    optimizer_lstm.step()

    return output, loss.item()

In [193]:
import time
import math
from torch.autograd import Variable

criterion_lstm = nn.NLLLoss()
rnn_lstm = RNN_LSTM(n_letters, n_hidden, n_categories)  
optimizer_lstm = torch.optim.Adam(rnn_lstm.parameters())

n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
train_loss_lstm = 0
all_train_losses_lstm = []
all_validation_losses_lstm = []
all_validation_losses2_lstm = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Just return an output given a line
def evaluate_lstm(line_tensor):
    hidden = rnn_lstm.initHidden()
    cell= rnn_lstm.initHidden()
    for i in range(line_tensor.size()[0]):
        output,hidden,cell = rnn_lstm(line_tensor[i], hidden,cell)
    return output

def eval_dataset_lstm(dataset):
    loss = 0
    n_instances = 0
    confusion = torch.zeros(n_categories, n_categories)
    for category in all_categories:
        category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
        n_instances += len(dataset[category])
        for line in dataset[category]:
            line_tensor = Variable(sentenceToTensor(line))
            output = evaluate_lstm(line_tensor)
            loss += criterion_lstm(output, category_tensor)
            guess, guess_i = categoryFromOutput(output)
            category_i = all_categories.index(category)
            confusion[category_i][guess_i] += 1

    # Normalize by dividing every row by its sum
    for i in range(n_categories):
        confusion[i] = confusion[i] / confusion[i].sum()

    return loss.item() / n_instances, confusion
  
print('\nIter \tTrain% \tTime \t\tTrain_loss_lstm \tExample')
start = time.time()
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train_LSTM(category_tensor, line_tensor)
    train_loss_lstm += loss

    # Print iter number, train loss average, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d \t%d%% \t(%s) \t%.4f \t\t%s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), train_loss_lstm / plot_every, line, guess, correct))

    # Add current train loss average to list of losses
    if iter % plot_every == 0:
        all_train_losses_lstm.append(train_loss_lstm / plot_every)
        train_loss_lstm = 0
        
    # Compute loss based on validation data
    if iter % plot_every == 0:
        average_validation_loss_lstm, _ = eval_dataset_lstm(val_data)

        # save model with best validation loss
        if len(all_validation_losses_lstm) == 0 or average_validation_loss_lstm < min(all_validation_losses_lstm):
            torch.save(rnn_lstm, 'char_rnn_lstm_classification_model.pt')
        all_validation_losses_lstm.append(average_validation_loss_lstm)



Iter 	Train% 	Time 		Train_loss_lstm 	Example


/srv/jupyter_python3-extras/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN_LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


5000 	5% 	(2m 52s) 	1.3251 		Reid & Taylor Men Edward Wallet / Wallets ✓
10000 	10% 	(6m 10s) 	1.0532 		United Colors of Benetton Men Check Brown Wht/purple / Headwear ✗ (Ties)
15000 	15% 	(9m 29s) 	0.8467 		Pieces Elements Mauve Ethnic Belt / Belts ✓
20000 	20% 	(12m 46s) 	0.7109 		Colorbar Color Intense Mignight Sky Liquid Eye Liner 001 / Makeup ✓
25000 	25% 	(16m 3s) 	0.6503 		Allen Solly Men Leather Black Wallet / Wallets ✓
30000 	30% 	(19m 22s) 	0.5749 		Puma Men Diet Cap / Headwear ✓
35000 	35% 	(22m 39s) 	0.5586 		ADIDAS Unisex Pack of Generation Crew Socks / Socks ✓
40000 	40% 	(25m 59s) 	0.4961 		Shree Women Navy & Burberry Salwar Suit with Dupatta / Apparel Set ✓
45000 	45% 	(29m 16s) 	0.4236 		Wills AH218-C1 Reespark Magenta Leggings / Bottomwear ✓
50000 	50% 	(32m 33s) 	0.4604 		Lakme Tshirts On Aqua Marine Eye Color / Makeup ✓
55000 	55% 	(35m 50s) 	0.4532 		Numero Uno Men's Navy Blue Shoe / Shoes ✓
60000 	60% 	(39m 7s) 	0.4122 		Parx Men Blue Tie / Ties ✓
65000 	65% 	(42m

In [195]:
model_LSTM = torch.load('char_rnn_lstm_classification_model.pt')

In [196]:
def predict(input_line, n_predictions=1):
    #print('\n> %s' % input_line)
    with torch.no_grad():
        output = torch.exp(evaluate_lstm(sentenceToTensor(input_line)))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            #print(all_categories[category_index])
            predictions.append(all_categories[category_index])
    return(output, predictions)
predict('women white winter casual')


(tensor([[4.6963e-06, 8.7542e-05, 7.7517e-04, 3.8517e-04, 2.2609e-03, 2.9723e-09,
          4.7479e-06, 9.7614e-05, 6.0423e-05, 1.3140e-04, 3.7311e-04, 3.4706e-08,
          8.8298e-01, 3.8908e-05, 1.7938e-04, 4.4881e-02, 1.2076e-04, 1.0829e-08,
          2.0813e-02, 6.6573e-06, 7.7208e-07, 1.6457e-02, 4.3971e-05, 8.7415e-07,
          2.8190e-02, 5.7435e-04, 1.5330e-03]]),
 ['Innerwear'])

In [197]:
test['combine_text']=test['noisyTextDescription'].astype(str)
test['combine_text']

0                        Chromozome Men Black Fashion Vest
1                         Elle Women Green Color Clash Top
2                     Baggit Women Chotu Mayur Black Palms
3        Greensboro Colors Of Ap Men Latnam Black Casua...
4                          Fish White Smooth daddy (SH100)
                               ...                        
21623                  Park ClubLife 602 Helium Sunglasses
21624                Vero Moda Women Black Rey Shift Dress
21625                     Nike Men Hell With White Tshirts
21626         Lakme Holi to Five Day Apricot Nectar Lip K9
21627                    Percorso Unisex Red Grit Backpack
Name: combine_text, Length: 21628, dtype: object

What is training accuracy? validation accuracy ?

In [198]:
#training accuracy:
pred_train_lstm=[]
label_train_lstm=[]

accu=0

for i in range(len(training_data)):
        output,pred=predict(training_data[i],n_predictions=1)
        #print(pred[0])
        if pred[0]==training_y[i]:
            accu+=1
        pred_train_lstm.append(np.array(output[0]))
        label_train_lstm.append(pred[0])

print(accu/len(training_data))
label_train_lstm[:10]


0.8131429892497977


['Scarves',
 'Flip Flops',
 'Topwear',
 'Sandal',
 'Belts',
 'Innerwear',
 'Socks',
 'Topwear',
 'Topwear',
 'Topwear']

In [199]:
#validation accuracy:
pred_val_lstm=[]
label_val_lstm=[]
accu=0

for i in range(len(validate_data)):
        output,pred=predict(validate_data[i],n_predictions=1)
        if pred[0]==validate_y[i]:
            accu+=1
        pred_val_lstm.append(np.array(output[0]))
        label_val_lstm.append(pred[0])

print(accu/len(validate_data))
label_val_lstm[:10]

0.7567630057803468


['Saree',
 'Eyewear',
 'Belts',
 'Topwear',
 'Headwear',
 'Bottomwear',
 'Topwear',
 'Bags',
 'Flip Flops',
 'Dress']

In [200]:
pred_test_lstm=[]
for line in test['combine_text']:
    output,pred=predict(line,n_predictions=1)
    pred_test_lstm.append(np.array(output[0]))

Ensembled learning:

**3)Use simple neural network to concatenate results from RNN and CNN, and output into single probability as final result:**

In [201]:
pred_train_lstm=np.array(pred_train_lstm)
pred_val_lstm=np.array(pred_val_lstm)
pred_test_lstm=np.array(pred_test_lstm)

In [206]:
y_val.shape

(4325, 27)

In [207]:
import keras
from keras.layers import Input, concatenate,  Dense
from keras.models import Model


# # Define two input layers
LSTM_input = Input((27,))
CNN_input = Input((27,))
 # create the input to our final set of layers as the *output* of both
 # the MLP and CNN
combinedInput = concatenate([LSTM_input, CNN_input])
## our final FC layer head will have two dense layers, the final one
 # being our regression head
x = Dense(27, activation="softmax")(combinedInput)
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[LSTM_input, CNN_input], outputs=x)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 27)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 27)           0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 54)           0           input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 27)           1485        concatenate_2[0][0]        

In [208]:
from keras import losses

#opt = keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# train the model
print("[INFO] training model...")
model.fit(x=[pred_train_lstm, prd_train], y=y_train,
         validation_data=([pred_val_lstm, prd_val], y_val),
           epochs=1)

[INFO] training model...
Train on 17302 samples, validate on 4325 samples
Epoch 1/1
17302/17302 [==============================] - 0s 24us/step - loss: 3.2824 - accuracy: 0.0575 - val_loss: 3.1888 - val_accuracy: 0.1427


In [209]:
torch.save(model, 'ensembled_model.pt')

In [210]:
model_ensemble=torch.load('ensembled_model.pt')

Write a csv file out which is corpus containing line and category separated by tab:

In [212]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict([pred_test_lstm, prd_test])
prd_y = np.argmax(preds, axis=1)
label = list(le.inverse_transform(prd_y))

predict=np.stack((test['id'],np.array(label)),axis=1)
import csv
with open('submission_finalresult.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['id','category'])
    writer.writerows(predict)

[INFO] predicting house prices...
